# Re-create your own _One Hot Encoder_ 

In [1]:
import pandas as pd
import seaborn as sns

## (1) The Titanic Dataset

In [2]:
# Loading 100% of the dataset. 
# Choose 0.5 to load only 50% of the rows randomly

data = sns.load_dataset('titanic').sample(frac = 1) 
data.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
884,0,3,male,25.0,0,0,7.050,S,Third,man,True,NaN,Southampton,no,True
659,0,1,male,58.0,0,2,113.275,C,First,man,True,D,Cherbourg,no,False
230,1,1,female,35.0,1,0,83.475,S,First,woman,False,C,Southampton,yes,False
465,0,3,male,38.0,0,0,7.050,S,Third,man,True,NaN,Southampton,no,True
21,1,2,male,34.0,0,0,13.000,S,Second,man,True,D,Southampton,yes,True


In [3]:
from sklearn.model_selection import train_test_split

X = data.drop(columns = ['survived', 'alive', 'who', 'adult_male'])
y = data['survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [4]:
X_train

,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
774,2,female,54.0,1,3,23.0000,S,Second,NaN,Southampton,False
606,3,male,30.0,0,0,7.8958,S,Third,NaN,Southampton,True
833,3,male,23.0,0,0,7.8542,S,Third,NaN,Southampton,True
587,1,male,60.0,1,1,79.2000,C,First,B,Cherbourg,False
514,3,male,24.0,0,0,7.4958,S,Third,NaN,Southampton,True
...,...,...,...,...,...,...,...,...,...,...,...
382,3,male,32.0,0,0,7.9250,S,Third,NaN,Southampton,True
556,1,female,48.0,1,0,39.6000,C,First,A,Cherbourg,False
98,2,female,34.0,0,1,23.0000,S,Second,NaN,Southampton,False
423,3,female,28.0,1,1,14.4000,S,Third,NaN,Southampton,False


## (2) A first pipeline

❓ Create a basic Pipeline which ***encodes categorical features*** and ***scales numerical features*** ❓

💡 Use [`make_pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html) and [`make_column_transformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.make_column_transformer.html)

In [5]:
num_features = ['age','fare','sibsp','parch']
cat_features = ['pclass','sex','embarked','class','embark_town','alone']

In [9]:
from sklearn import set_config
set_config(display="diagram")

from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_transformer = make_pipeline(
    SimpleImputer(strategy='median'),
    StandardScaler()
)

cat_transformer = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OneHotEncoder(handle_unknown='ignore')
)

preprocessor = make_column_transformer(
    (num_transformer, num_features),
    (cat_transformer, cat_features)
)

preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['age', 'fare', 'sibsp', 'parch']),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['pclass', 'sex', 'embarked', 'class',
                                  'embark_town', 'alone'])])

In [10]:
pd.DataFrame(preprocessor.fit_transform(X_train))

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,1.884309,-0.178650,0.480524,3.287006,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.053122,-0.503872,-0.488300,-0.461264,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,-0.480974,-0.504768,-0.488300,-0.461264,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
3,2.342105,1.031442,0.480524,0.788159,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,-0.404675,-0.512485,-0.488300,-0.461264,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618,0.205721,-0.503243,-0.488300,-0.461264,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
619,1.426512,0.178779,0.480524,-0.461264,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
620,0.358320,-0.178650,-0.488300,0.788159,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
621,-0.099477,-0.363824,0.480524,0.788159,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0


<details>
    <summary>👩🏻‍🏫 <i>Pipeline</i> vs. <i>make_pipeline</i></summary>

* When you create a Pipeline with `Pipeline()`, you have to:
    - specify all the ***sequential steps of the pipeline*** in a list
    - each step is a tuple with:
        - "name_of_the_step"
        - official Scikit-Learn name of the step
    
```python
Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
```
  
* When you create a Pipeline with `make_pipeline()`,
    - you don't have give a name to each step
    - you can simply chain all the steps together using their official Scikit-Learn name
    - the names of the steps are automatically induced by `make_pipeline`
    
```python
make_pipeline(
    SimpleImputer(strategy='median'),
    StandardScaler()
)
```
    
</details>

<details>
    <summary>👩🏻‍🏫 <i>ColumnTransformer</i> vs. <i>make_column_transformer</i></summary>

* When you create a ColumnTransformer with `ColumnTransformer()`, you have to:
    - specify all the ***parallel steps of the columns' transformer*** in a list
    - each step is a tuple with:
        - "name_of_the_transformer"
        - the transformer
        - the columns which will be impacted by the transformer
    
```python
ColumnTransformer([
    ('num_transformer', num_transformer, num_features),
    ('cat_transformer', cat_transformer, cat_features)
])
```
  
* When you create a ColumnTransformer with `make_column_transformer()`,
    - you don't have give a name to each parallel step
    - each step is a tuple with:
        - the transformer
        - the columns which will be impacted by the transformer
    
```python
make_column_transformer(
    (num_transformer, num_features),
    (cat_transformer, cat_features)
)
```
    
</details>

❓ Chain this preprocessing pipeline with a classifier and optimize it ❓

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats

final_pipe = make_pipeline(
    preprocessor,
    LogisticRegression(penalty='l1', solver='liblinear')
)

# If you are not sure about which params of your pipeline to optimize...
# ... remember that you can access them with final_pipe.get.params()

search = RandomizedSearchCV(final_pipe,
                            param_distributions={
                                'logisticregression__C': stats.loguniform(0.01,100), 
                            },
                            cv=3, 
                            scoring="accuracy", 
                            n_iter=20, 
                            n_jobs=-1)

search.fit(X_train,y_train);

❓ What are the best params and the best score ❓

In [12]:
search.best_params_

{'logisticregression__C': 2.8697424354507537}

In [13]:
search.best_score_

0.7993156199677939

## (3) How could we design a Custom Encoder to keep track of the columns' names?

In [14]:
# By default, OneHotEncoder works with Numpy and loses track of columns' names...
ohe = OneHotEncoder(sparse=False)
ohe.fit_transform(X_train[['sex']])

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [0., 1.]])

In [15]:
# ... however, we can access the one-hot-encoded names as follows
ohe.get_feature_names_out()

array(['sex_female', 'sex_male'], dtype=object)

❓ Try to create your own OneHotEncoder so that it preserves the columns names ❓

### Wiser - Update the OneHotEncoder Class

In [16]:
from sklearn.preprocessing import OneHotEncoder


class CustomOHE1(OneHotEncoder):

    def transform(self, *args, **kwargs):
        return pd.DataFrame(super().transform(*args, **kwargs), columns=self.get_feature_names_out())

In [17]:
c = CustomOHE1(sparse=False)
c.fit(X_train[['sex']])
c.transform(X_train[['sex']])
c.fit_transform(X_train[['sex']]).head()

,sex_female,sex_male
0,1.0,0.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0


### Harder option : recode the OneHotEmcoder using _get_dummies_ from _pandas_

In [18]:
from sklearn.base import TransformerMixin, BaseEstimator


class CustomOHE2(TransformerMixin, BaseEstimator):

    def __init__(self):
        pass

    def fit(self, X):
        X_dummified = pd.get_dummies(X)
        self.columns = X_dummified.columns
        return self

    def transform(self, X):
        X_dummified = pd.get_dummies(X)
        # Only keep columns that are computed in the fit() method
        # Drop new dummy columns if new category appears in the test set that were never seen in train set
        X_dummified_reindexed = X_dummified.reindex(columns=self.columns, fill_value=0)
        return X_dummified_reindexed

In [19]:
CustomOHE2().fit(X_train[['sex']]).transform(X_train[['sex']])
CustomOHE2().fit_transform(X_train[['sex']]).head()

,sex_female,sex_male
774,1,0
606,0,1
833,0,1
587,0,1
514,0,1


In [20]:
# For the Pipeline, it's working as expected and returns columns' names
pipeline = make_pipeline(CustomOHE1(sparse=False))

pd.DataFrame(pipeline.fit_transform(X_train[['sex']])).head()

,sex_female,sex_male
0,1.0,0.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0


**⚠️ But then again, we lose columns' names when passing `CustomOHE1` into a ColumnTransformer!** 👇

In [21]:
preprocessor = make_column_transformer(
    (CustomOHE1(sparse=False), ['sex'])
)

pd.DataFrame(preprocessor.fit_transform(X_train)).head()

,0,1
0,1.0,0.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0


**🤯🤯🤯 We also have to recode the ColumnTransformer ourself!**  
Good news, our default OneHot should pass the column names now!

In [22]:
from sklearn.compose import ColumnTransformer

class CustomColumnTransformer(ColumnTransformer):
    def transform(self, *args, **kwargs):
        return pd.DataFrame(super().transform(*args, **kwargs), columns=self.get_feature_names_out())
    def fit_transform(self, *args, **kwargs):
        return pd.DataFrame(super().fit_transform(*args, **kwargs), columns=self.get_feature_names_out())

In [23]:
preprocessor = CustomColumnTransformer([
    ('ohe', OneHotEncoder(sparse=False), ['sex']),
    ('custom_ohe', CustomOHE1(sparse=False), ['sex'])
])
preprocessor.fit(X_train)
preprocessor.transform(X_train)
preprocessor.fit_transform(X_train).head()

,ohe__sex_female,ohe__sex_male,custom_ohe__sex_female,custom_ohe__sex_male
0,1.0,0.0,1.0,0.0
1,0.0,1.0,0.0,1.0
2,0.0,1.0,0.0,1.0
3,0.0,1.0,0.0,1.0
4,0.0,1.0,0.0,1.0


🏁 If you want to build a very advanced pipeline, feel free to explore the Optional Challenge dealing the `cars dataset` !

💾 Don't forget to git add/commit/push your notebook.

👏 Congratulations, you are now a master at Pipeline and ColumnTransformer.